In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://ppa

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [5]:
# filter to retrieve rows of total_votes >=20
vote_count_df = df.filter("total_votes >= 20")
vote_count_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|   35346149| R5O4WE9UM60B0|B00Q8BYTAW|      48877050|Women's New Simpl...|         Jewelry|          1|    

In [6]:
# filter to retrieve rows helpful_votes divided by total_votes >= 50%
helpful_votes_df = vote_count_df.filter(vote_count_df["helpful_votes"]/vote_count_df["total_votes"]>=0.5)
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|   35346149| R5O4WE9UM60B0|B00Q8BYTAW|      48877050|Women's New Simpl...|         Jewelry|          1|    

In [8]:
#retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'Y')
paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51635935|R15PXA9XU9ZKSU|B00TDNZV4A|     280424829|Sterling Silver C...|         Jewelry|          4|           29|         29|   Y|                N|So classy and gor...|You can wear thes...| 2015-05-09|
|         US|   52833354|R2LJ4QYQ4PYDEH|B00TDNZUYG|     280424829|Sterling Silver C...|         Jewelry|          5|    

In [9]:
#retrieves all the rows where a review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'N')
unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|   35346149| R5O4WE9UM60B0|B00Q8BYTAW|      48877050|Women's New Simpl...|         Jewelry|          1|    

In [19]:
# determine the number of five star reviews of paid_df
five_star_paid_df = paid_df.filter(paid_df['star_rating'] == 5)
five_star_paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52833354|R2LJ4QYQ4PYDEH|B00TDNZUYG|     280424829|Sterling Silver C...|         Jewelry|          5|           24|         26|   Y|                N|           Stunning!|I was very pleasa...| 2015-05-06|
|         US|   28779404|R2PJWWQZ37WCIC|B00S5VKNIA|     940533137|Sterling Silver F...|         Jewelry|          5|    

In [20]:
# determine the number of five star reviews of unpaid_df
five_star_unpaid_df = unpaid_df.filter(unpaid_df['star_rating'] == 5)
five_star_unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|    4039595|R2OKV47GETH0L7|B00VUTQVDK|     939458680|Jewelrieshop Stai...|         Jewelry|          5|    

In [18]:
#total number of reviews
vote_count_df.count()

7962

In [23]:
#number of vine reviews
paid_df.count()

21

In [24]:
#number of non-vine reviews
unpaid_df.count()

7689

In [12]:
#the number of 5-star reviews of paid
five_satar_paid_df.count()

11

In [13]:
#the number of 5-star reviews of unpaid
five_satar_unpaid_df.count()

4444

In [21]:
#percentage of Vine reviews were 5 stars
five_satar_paid_df.count()/paid_df.count()

0.5238095238095238

In [22]:
#percentage of non-Vine reviews were 5 stars
five_satar_unpaid_df.count()/unpaid_df.count()


0.5779685264663805